# Introduction to `awesimsoss`

**Advanced Webb Exposure Simulator for SOSS**

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import astropy.units as q
import astropy.constants as ac
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
import batman
from pkg_resources import resource_filename
from awesimsoss import awesim

output_notebook()

## M Dwarf (no planet)
Here is how to generate time series observations of a brown dwarf (or any other isolated star with no transiting planet).

We need two components to generate this simulation:
- A flux calibrated stellar spectrum
- A specified number of integrations and groups for the observation

Let's use this 3500K stellar spectrum with a J magnitude of 9.

In [ ]:
# Get the wavelength and flux of the star with units 
star = np.genfromtxt(resource_filename('awesimsoss','files/scaled_spectrum.txt'), unpack=True)
star_wave, star_flux = [star[0][:1000]*q.um, (star[1][:1000]*q.W/q.m**2/q.um).to(q.erg/q.s/q.cm**2/q.AA)]  

# Plot it
fig = figure(width=600, height=300, title='3500 K star with Jmag=9')
fig.line(star_wave, star_flux, legend='Input Spectrum')
fig.xaxis.axis_label = 'Wavelength [um]'
fig.yaxis.axis_label = 'Flux Density [erg/s/cm3/A]'
show(fig)

Now we can intialize the simulation by passing the number of groups (3) and integrations (10) along with the stellar spectrum to the `TSO` class.

In [ ]:
# Initialize the simulation with 3 groups and 10 integrations
my_TSO = awesim.TSO(ngrps=2, nints=2, star=[star_wave, star_flux], target='WASP-107')  

In [ ]:
# Run the simulation (takes ~10 seconds)
my_TSO.simulate()

We can view frames of the simulation with the `plot` method like so:

In [ ]:
# plot the TSO object 
my_TSO.plot()  

## M Dwarf (with planet)
Let's pretend this M dwarf is orbited by WASP107b! Why not? First get the transmission spectrum:

In [ ]:
# Get the planet data
planet = np.genfromtxt(resource_filename('awesimsoss', '/files/WASP107b_pandexo_input_spectrum.dat'), unpack=True)
planet_wave, planet_trans = [planet[0]*q.um, planet[1]]

# Plot it
fig = figure(width=600, height=300, title='Planetary Transit Spectrum')
fig.line(planet_wave, planet_trans, legend='Input Transmission')
fig.xaxis.axis_label = 'Wavelength [um]'
fig.yaxis.axis_label = 'Transit Depth [ppm]'
show(fig)

In [ ]:
# Set the orbital parameters with the Batman package (https://www.cfa.harvard.edu/~lkreidberg/batman/quickstart.html)
params = batman.TransitParams()
params.t0 = 0.001               # Time of inferior conjunction (days)
params.per = 0.03               # Orbital period (days)
params.rp = 0.15                # Planet radius (in units of R*)
params.a = 0.0558*q.AU.to(ac.R_sun)*0.66                  # Semi-major axis (in units of R*)
params.inc = 89.8               # Orbital inclination (in degrees)
params.ecc = 0.                 # Eccentricity
params.w = 90.                  # Longitude of periastron (in degrees) 
params.u = [0.1, 0.1]           # Limb darkening coefficients [u1, u2]
params.limb_dark = "quadratic"  # Limb darkening model

# Make the transit model and add the stellar params
tmodel = batman.TransitModel(params, my_TSO.time.jd)
tmodel.teff = 3500              # Effective temperature of the host star
tmodel.logg = 5                 # log surface gravity of the host star
tmodel.feh = 0                  # Metallicity of the host star

In [ ]:
# Run the simulation, this time including the planet
my_TSO.simulate(planet=[planet_wave, planet_trans], tmodel=tmodel)

## Exporting Results

Create a fits file with your time series observations with ``export`` like so:

In [ ]:
my_TSO.export('my_soss_simulation.fits')